In [1]:
import torch
import csv
import numpy as np
from torch.utils.data import DataLoader
from torch.utils.data import TensorDataset
from torch import nn
device = "cuda" if torch.cuda.is_available() else "cpu"
torch.cuda.set_device(1)

In [2]:
root = "C:/Users/sjw00/OneDrive/Desktop/dataset/"

temp_list = list()
r=open (root+'mnist_data/training_input.csv', 'r' )
reader=csv.reader(r)
for target in reader:
    temp_list.append(target)
training_data = np.array(temp_list).reshape((-1, 1, 28, 28)).astype(np.float32)
print(training_data.shape)
training_data = torch.from_numpy(training_data).to("cuda")

temp_list = list()
r=open (root+'mnist_data/test_input.csv', 'r' )
reader=csv.reader(r)
for target in reader:
    temp_list.append(target)
test_data = np.array(temp_list).reshape((-1, 1, 28, 28)).astype(np.float32)
print(test_data.shape)
test_data = torch.from_numpy(test_data).to("cuda")

temp_list = list()
r=open (root+'mnist_data/training_target.csv', 'r' )
reader=csv.reader(r)
for target in reader:
    if target[0] == '0':
        temp_list.append([1,0,0,0,0,0,0,0,0,0])
        continue
    if target[0] == '1':
        temp_list.append([0,1,0,0,0,0,0,0,0,0])
        continue
    if target[0] == '2':
        temp_list.append([0,0,1,0,0,0,0,0,0,0])
        continue
    if target[0] == '3':
        temp_list.append([0,0,0,1,0,0,0,0,0,0])
        continue
    if target[0] == '4':
        temp_list.append([0,0,0,0,1,0,0,0,0,0])
        continue
    if target[0] == '5':
        temp_list.append([0,0,0,0,0,1,0,0,0,0])
        continue
    if target[0] == '6':
        temp_list.append([0,0,0,0,0,0,1,0,0,0])
        continue
    if target[0] == '7':
        temp_list.append([0,0,0,0,0,0,0,1,0,0])
        continue
    if target[0] == '8':
        temp_list.append([0,0,0,0,0,0,0,0,1,0])
        continue
    if target[0] == '9':
        temp_list.append([0,0,0,0,0,0,0,0,0,1])
        continue 
training_target = np.array(temp_list).astype(np.float32)
print(training_target.shape)
training_target = torch.from_numpy(training_target).to("cuda")

temp_list = list()
r=open (root+'mnist_data/test_target.csv', 'r' )
reader=csv.reader(r)
for target in reader:
    if target[0] == '0':
        temp_list.append([1,0,0,0,0,0,0,0,0,0])
        continue
    if target[0] == '1':
        temp_list.append([0,1,0,0,0,0,0,0,0,0])
        continue
    if target[0] == '2':
        temp_list.append([0,0,1,0,0,0,0,0,0,0])
        continue
    if target[0] == '3':
        temp_list.append([0,0,0,1,0,0,0,0,0,0])
        continue
    if target[0] == '4':
        temp_list.append([0,0,0,0,1,0,0,0,0,0])
        continue
    if target[0] == '5':
        temp_list.append([0,0,0,0,0,1,0,0,0,0])
        continue
    if target[0] == '6':
        temp_list.append([0,0,0,0,0,0,1,0,0,0])
        continue
    if target[0] == '7':
        temp_list.append([0,0,0,0,0,0,0,1,0,0])
        continue
    if target[0] == '8':
        temp_list.append([0,0,0,0,0,0,0,0,1,0])
        continue
    if target[0] == '9':
        temp_list.append([0,0,0,0,0,0,0,0,0,1])
        continue 
test_target = np.array(temp_list).astype(np.float32)
print(test_target.shape)
test_target = torch.from_numpy(test_target).to("cuda")


(60000, 1, 28, 28)
(10000, 1, 28, 28)
(60000, 10)
(10000, 10)


In [3]:
batch_size = 100
epochs = 500


In [4]:
training_dataset = TensorDataset(training_data, training_target)
test_dataset = TensorDataset(test_data, test_target)

training_dataloader = DataLoader(training_dataset, batch_size=batch_size, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

In [1]:
#Pytorch 2.5.1
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        
        self.layer_1 = nn.Conv2d(1,1,3,stride=1,padding=1)
        self.layer_2 = nn.Conv2d(1,1,3,stride=1,padding=1)
        self.layer_3 = nn.Conv2d(1,1,3,stride=1,padding=1)
        self.layer_4 = nn.Linear(784,10)
        self.lrelu = nn.LeakyReLU()
        self.flatten = nn.Flatten()
        self.softmax = nn.Softmax(dim=1)
        
        torch.nn.init.zeros_(self.layer_1.weight)
        torch.nn.init.zeros_(self.layer_1.bias)
        torch.nn.init.zeros_(self.layer_2.weight)
        torch.nn.init.zeros_(self.layer_2.bias)
        torch.nn.init.zeros_(self.layer_3.weight)
        torch.nn.init.zeros_(self.layer_3.bias)
        torch.nn.init.zeros_(self.layer_4.weight)
        torch.nn.init.zeros_(self.layer_4.bias)
        
    def forward(self, x):
        x_1 = self.lrelu(self.layer_1(x)) + x
        x_1 = self.lrelu(self.layer_2(x_1)) + x_1
        x_1 = self.lrelu(self.layer_3(x_1)) + x_1
        x_1 = self.flatten(x_1)
        x_1 = self.layer_4(x_1)
        return x_1
    

Total number of parameters: 7880


In [6]:
def train(dataloader, model, loss_fn, optimizer, scaler):   
    for batch, (X, y) in enumerate(dataloader):
        with torch.autocast(device_type='cuda', dtype=torch.bfloat16):
            pred = model(X)
            batch_loss_result = loss_fn(pred, y)
        optimizer.zero_grad()

        scaler.scale(batch_loss_result).backward() # Backpropagation: compute gradients
        scaler.step(optimizer) # Update model parameters using the optimizer
        scaler.update() # Update the gradient scaler

    


In [7]:
def test(dataloader, model):
    with torch.no_grad():
        accuracy_sum=0
        
        for _, (X, y) in enumerate(dataloader):
            with torch.autocast(device_type='cuda', dtype=torch.bfloat16):
                pred = model(X)
            accuracy_sum+= (torch.argmax(pred, dim=1) == torch.argmax(y,dim=1)).type(torch.float).sum().item()
            

        print("test_accuracy", accuracy_sum/10000)
    return accuracy_sum/10000

In [8]:
summary = list()
total_list = []
for i in range(20):
    model = CNN().to("cuda")
    loss_fn = nn.CrossEntropyLoss()
    scaler = torch.amp.GradScaler()
    optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)
    scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=20, gamma=0.95)
    accuracy = 0 
    epoch_list = []
    for t in range(epochs):
        print(f"Epoch {t+1}\n-------------------------------")
        train(training_dataloader, model, loss_fn, optimizer, scaler)
        accuracy_temp = test(test_dataloader, model)
        epoch_list.append(accuracy_temp)
        scheduler.step()
        if accuracy_temp > accuracy:
            accuracy = accuracy_temp
    total_list.append(epoch_list)
    summary.append(accuracy)
    print(summary)
with open("mnist_allzero.csv", mode="w", newline="", encoding="utf-8") as file:
    writer = csv.writer(file)
    writer.writerows(total_list)
print("Done!")
print(summary)

Epoch 1
-------------------------------
test_accuracy 0.9038
Epoch 2
-------------------------------
test_accuracy 0.9028
Epoch 3
-------------------------------
test_accuracy 0.9033
Epoch 4
-------------------------------
test_accuracy 0.9052
Epoch 5
-------------------------------
test_accuracy 0.8974
Epoch 6
-------------------------------
test_accuracy 0.9059
Epoch 7
-------------------------------
test_accuracy 0.9067
Epoch 8
-------------------------------
test_accuracy 0.9106
Epoch 9
-------------------------------
test_accuracy 0.9011
Epoch 10
-------------------------------
test_accuracy 0.9164
Epoch 11
-------------------------------
test_accuracy 0.9117
Epoch 12
-------------------------------
test_accuracy 0.9074
Epoch 13
-------------------------------
test_accuracy 0.9131
Epoch 14
-------------------------------
test_accuracy 0.9028
Epoch 15
-------------------------------
test_accuracy 0.9036
Epoch 16
-------------------------------
test_accuracy 0.8949
Epoch 17
--------